In [1]:
pip install mysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mysql.connector
from mysql.connector import Error

try:
    # Connect to the MySQL database
    conn = mysql.connector.connect(
        host='127.0.0.1',    # Replace with your host
        user='root',         # Replace with your username
        password='Admin@123', # Replace with your password
        database='eduonix',  # Replace with your database name
        port=3306            # Default MySQL port
    )

    if conn.is_connected():
        print("Connected to the database")
    
    cur = conn.cursor()

    # Create the Counts table
    cur.execute('''
    DROP TABLE IF EXISTS Counts''')

    cur.execute('''
    CREATE TABLE Counts (org VARCHAR(255), count INTEGER)''')

    # Function to extract the domain from an email address
    def get_domain(email):
        try:
            return email.split('@')[1]
        except IndexError:
            return None

    # Read the mbox.txt file
    file_name = 'mbox.txt'
    try:
        fhand = open(file_name)
    except:
        print(f"File cannot be opened: {file_name}")
        quit()

    # Dictionary to store domain counts
    counts = {}

    # Process each line in the file
    for line in fhand:
        if line.startswith('From: '):
            email = line.split()[1]
            domain = get_domain(email)
            if domain:
                counts[domain] = counts.get(domain, 0) + 1

    # Insert or update counts in the database
    for domain, count in counts.items():
        cur.execute('SELECT count FROM Counts WHERE org = %s', (domain,))
        row = cur.fetchone()
        if row is None:
            cur.execute('''INSERT INTO Counts (org, count)
                    VALUES (%s, %s)''', (domain, count))
        else:
            cur.execute('UPDATE Counts SET count = count + %s WHERE org = %s', (count, domain))

    # Commit the changes to the database
    conn.commit()

    # Print the counts
    cur.execute('SELECT org, count FROM Counts ORDER BY count DESC')

    print("Counts:")
    for row in cur:
        print(str(row[0]), row[1])

    # Print the table structure
    print("\nTable Structure:")
    cur.execute('DESCRIBE Counts')
    for row in cur.fetchall():
        print(row)

    # Print the table content
    print("\nTable Content:")
    cur.execute('SELECT * FROM Counts')
    for row in cur.fetchall():
        print(row)

except Error as e:
    print(f"Error: {e}")
finally:
    if conn.is_connected():
        cur.close()
        conn.close()
        print("Connection closed")


Connected to the database
Counts:
iupui.edu 8
umich.edu 7
uct.ac.za 6
media.berkeley.edu 4
caret.cam.ac.uk 1
gmail.com 1

Table Structure:
('org', 'varchar(255)', 'YES', '', None, '')
('count', 'int', 'YES', '', None, '')

Table Content:
('uct.ac.za', 6)
('media.berkeley.edu', 4)
('umich.edu', 7)
('iupui.edu', 8)
('caret.cam.ac.uk', 1)
('gmail.com', 1)
Connection closed
